In [ ]:
import os

import pandas as pd
import numpy as np

import geopandas as gpd
import folium

import matplotlib.pyplot as plt

In [ ]:
# function to create heatmap
def create_heatmap(geo_data, data, id_col, value_col, legend_name, center = [-37.8136, 144.9631], zoom_start=10, colormap='YlOrRd'):
    """
    Plots a choropleth map using folium.

    Parameters:
    - geo_data: GeoDataFrame containing the geometry information
    - data: DataFrame containing the data to be visualized
    - id_col: Column in geo_data that matches the 'id' in the GeoJSON representation
    - value_col: Column in data that contains the values to be visualized
    - legend_name: Legend Name
    - zoom_start: Initial zoom level for the map
    - colormap: Color map for the choropleth visualization
    """
    
    # Create a base map
    m = folium.Map(location=center, zoom_start=zoom_start, tiles='cartodb positron')
    
    # Plot the choropleth
    folium.Choropleth(
        geo_data=geo_data.to_json(),
        data=data,
        columns=[id_col, value_col],
        key_on=f'feature.properties.{id_col}',
        fill_color=colormap,
        legend_name=legend_name,
        nan_fill_color="white",
        highlight=True
    ).add_to(m)
    
    return m

In [ ]:
# read in shapefile for visualisations
sf_SA2 = gpd.read_file("../../data/landing/SA2_2021_AUST_GDA2020.shp")

# taking only victoria borders
sf_SA2 = sf_SA2[sf_SA2['STE_NAME21'] == 'Victoria']

# convert to lowercase
sf_SA2['SA2_NAME21'] = sf_SA2['SA2_NAME21'].str.lower()

## Income Heatmap


In [ ]:
# read in data
df_inc = pd.read_csv('../../data/curated/curated_income.csv')

# merge dataframes
gdf_inc = sf_SA2.merge(df_inc, left_on = 'SA2_NAME21', right_on = 'sa2 name', how = 'left')

# plot chloropleth map
heatmap_inc = create_heatmap(geo_data = gdf_inc, data = gdf_inc, 
                                 id_col = 'SA2_NAME21', value_col = '2023', legend_name = 'Average Income (Projected 2023)')

heatmap_inc.save('../../plots/income_heatmap.html')
heatmap_inc

## Population Heatmap

In [ ]:
# read in data
df_pop = pd.read_csv('../../data/curated/curated_population.csv')

# merge dataframes
gdf_pop = sf_SA2.merge(df_pop, left_on = 'SA2_NAME21', right_on = 'sa2 name', how = 'left')

# plot chloropleth map
heatmap_pop = create_heatmap(geo_data = gdf_pop, data = gdf_pop, 
                                 id_col = 'SA2_NAME21', value_col = '2023', legend_name = 'Population (Projected 2023)')

heatmap_pop.save('../../plots/population_heatmap.html')
heatmap_pop

## Past Rental Price Distribution

In [ ]:
# read in data
df_rent = pd.read_csv('../../data/mapped/mapped_rent_av.csv')

# calculate top 10 most expensive on average
df_rent['mean_price'] = df_rent.iloc[:, 1:].mean(axis=1)
top10 = df_rent.sort_values(by='mean_price', ascending=False).head(10)

# plot
plt.figure(figsize = (16, 8))

# iterate through the top 10, one line at a time
for idx, row in top10.iterrows():
    plt.plot(row.index[1:-1], row.values[1:-1], label=row['sa2'], marker='o')
    
plt.title('Median weekly rent for the top 10 most expensive suburbs', fontsize=30, y=1.008)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Weekly Rent ($AUD)', fontsize=20)
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.xlabel('Year', fontsize=20)

plt.savefig('../../plots/suburb_rent_line.png')
plt.show()